<div align="center"><img src="asset/1613718500797.jpg" /></div>

# 参考文档
- [Spark Documentation](http://spark.apache.org/docs/latest/)
- [Spark Programming Guide](http://spark.apache.org/docs/latest/programming-guide.html)
- [Pyspark](https://github.com/jupyter/docker-stacks)
- [Conda](https://docs.conda.io/en/latest/)

# 安装实验环境

## 安装 Spark

下载 Spark 压缩包，[官方下载地址](http://spark.apache.org/downloads.html)，如果你下载不下来，可以使用我下载的版本。

链接: https://pan.baidu.com/s/1eTCybrBq-iWPeYMAhl1vHg  密码: 8peu

我们这里选用 Spark 3.0 的版本。

## 安装 PySpark

你可以使用以下命令来安装 pyspark。

```
pip install pyspark
pip install findspark
pip install happybase
```

或者

```
conda install -c conda-forge pyspark 
conda install findspark
pip install happybase
```

PySpark 的官方参考文档在[这里](https://spark.apache.org/docs/latest/api/python/),如果安装过程中出现什么问题，可以来这里找找。

运行下面的命令，看看你是否已经安装成功了。

In [2]:
import pyspark
import findspark

# 一个简单的测试

In [3]:
findspark.init()
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

3.14144332


# 概述

## Spark 是什么？

![1613718565213.jpg](./asset/1613718565213.jpg)



Spark 是一种基于内存的快速、通用、可扩展的大数据分析计算引擎。

## Spark 和 Hadoop 的区别


相信大家都听说过火的不能再火、炒得不能再炒的新一代大数据处理框架 Spark. 那么 Spark 是何方神圣？为何大有取代 Hadoop 的势头？Spark 内部又是如何工作的呢？我们会用几篇文章为大家一一介绍。

**Hadoop：我不想知道我是怎么来的，我就想知道我是怎么没的？**

还是从 Hadoop 处理海量数据的架构说起，一个 Hadoop job 通常都是这样的：

- 从 HDFS 读取输入数据；
- 在 Map 阶段使用用户定义的 mapper function, 然后把结果写入磁盘；
- 在 Reduce 阶段，从各个处于 Map 阶段的机器中读取 Map 计算的中间结果，使用用户定义的 reduce function, 通常最后把结果写回 HDFS; 

不知道大家是否注意到，一个 Hadoop job 进行了多次磁盘读写，比如写入机器本地磁盘，或是写入分布式文件系统中（这个过程包含磁盘的读写以及网络传输）。考虑到磁盘读取比内存读取慢了几个数量级，所以像 Hadoop 这样高度依赖磁盘读写的架构就一定会有性能瓶颈。

在实际应用中，我们通常需要设计复杂算法处理海量数据，比如之前提过的 Google crawling indexing ranking, 显然不是一个 Hadoop job 可以完成的。再比如现在风生水起的机器学习领域，大量使用迭代的方法训练机器学习模型。而像 Hadoop 的基本模型就只包括了一个 Map 和 一个 Reduce 阶段，想要完成复杂运算就需要切分出无数单独的 Hadoop jobs, 而且每个 Hadoop job 都是磁盘读写大户，这就让 Hadoop 显得力不从心。

随着业界对大数据使用越来越深入，大家都呼唤一个更强大的处理框架，能够真正解决更多复杂的大数据问题。

**Spark 横空出世**
2009年，美国加州大学伯克利分校的 AMPLab 设计并开发了名叫 Spark 的大数据处理框架。真如其名，Spark 像燎原之火，迅猛占领大数据处理框架市场。

**性能优秀**
Spark 没有像 Hadoop 一样使用磁盘读写，而转用性能高得多的内存存储输入数据、处理中间结果、和存储最终结果。在大数据的场景中，很多计算都有循环往复的特点，像 Spark 这样允许在内存中缓存输入输出，上一个 job 的结果马上可以被下一个使用，性能自然要比 Hadoop MapReduce 好得多。

同样重要的是，Spark 提供了更多灵活可用的数据操作，比如 filter, union, join, 以及各种对 key value pair 的方便操作，甚至提供了一个通用接口，让用户根据需要开发定制的数据操作。

此外，Spark 本身作为平台也开发了 streaming 处理框架 spark streaming, SQL 处理框架 Dataframe, 机器学习库 MLlib, 和图处理库 GraphX. 如此强大，如此开放，基于 Spark 的操作，应有尽有。

**Hadoop 的 MapReduce 为什么不使用内存存储？选择依赖 HDFS，岂不给了后来者据上的机会？**

历史原因。当初 MapReduce 选择磁盘，除了要保证数据存储安全以外，更重要的是当时企业级数据中心购买大容量内存的成本非常高，选择基于内存的架构并不现实；现在 Spark 真的赶上了好时候，企业可以轻松部署多台大内存机器，内存大到可以装载所有要处理的数据。

更加上这两年机器学习算法越来越丰富，越来越成熟，真正能够为企业解决问题。那当一个企业想要使用大数据处理框架时，是使用拿来就可以用、强大支持的 Spark ？还是去改进 Hadoop 的性能和通用性？显然是前者。

基于以上种种原因，我们看 Spark 被炒得这么火，也就不足为奇了。

**只要把 Hadoop 的 MapReduce 转为基于内存的架构不就解决了？听起来没有本质变化嘛。**

非也。MapReduce 定死了 Map 和 Reduce 两种运算以及之间 shuffle 的数据搬运工作。就是 Hadoop 运算无论多么灵活，你都要走 map -> shuffle -> reduce 这条路。要支持各类不同的运算，以及优化性能，还真不是改个存储介质这么简单。AMPLab 为此做了精心设计，让各种数据处理都能得心应手。

## Spark 和 Hadoop 对比

- 时间
    - Hadoop
        1. 2006 年 1 月，Doug Cutting 加入 Yahoo，领导 Hadoop 的开发
        2. 2008年1月，Hadoop成为Apache顶级项目
        3. 2011年1.0正式发布
        4. 2012年3月稳定版发布
        5. 2013 年 10 月发布 2.X (Y arn)版本
    - Spark
        1. 2009年，Spark诞生于伯克利大学的AMPLab实验室
        2. 2010年，伯克利大学正式开源了Spark项目
        3. 2013年6月，Spark成为了Apache基金会下的项目
        4. 2014年2月，Spark以飞快的速度成为了Apache的顶级项目
        5. 2015年至今，Spark变得愈发火爆，大量的国内公司开始重点部署或者使用Spark

    

- 功能
    - Hadoop
        1. Hadoop是由java语言编写的，在分布式服务器集群上存储海量数据并运行分布式分析应用的开源框架
        2. 作为 Hadoop 分布式文件系统，HDFS 处于 Hadoop 生态圈的最下层，存储着所有的数据，支持着 Hadoop 的所有服务。它的理论基础源于 Google 的TheGoogleFileSystem 这篇论文，它是 GFS 的开源实现。
        3. MapReduce是一种编程模型，Hadoop根据Google的MapReduce论文将其实现， 作为 Hadoop 的分布式计算模型，是 Hadoop 的核心。基于这个框架，分布式并行 程序的编写变得异常简单。综合了 HDFS 的分布式存储和 MapReduce 的分布式计 算，Hadoop 在处理海量数据时，性能横向扩展变得非常容易。
        4. HBase是对Google的Bigtable的开源实现，但又和Bigtable存在许多不同之处。 HBase 是一个基于 HDFS 的分布式数据库，擅长实时地随机读/写超大规模数据集。 它也是 Hadoop 非常重要的组件。
    - Spark
        1. Spark是一种由Scala语言开发的快速、通用、可扩展的大数据分析引擎
        2. SparkCore中提供了Spark最基础与最核心的功能
        3. Spark SQL是Spark用来操作结构化数据的组件。通过Spark SQL，用户可以使用 SQL 或者 Apache Hive 版本的 SQL 方言(HQL)来查询数据。
        4. Spark Streaming 是 Spark 平台上针对实时数据进行流式计算的组件，提供了丰富的处理数据流的 API。

## 使用 Spark 还是 Hadoop ？

Hadoop 的 MR 框架和 Spark 框架都是数据处理框架，那么我们在使用时如何选择呢?

- Hadoop MapReduce 由于其设计初衷并不是为了满足循环迭代式数据流处理，因此在多 并行运行的数据可复用场景(如:机器学习、图挖掘算法、交互式数据挖掘算法)中存 在诸多计算效率等问题。所以 Spark 应运而生，Spark 就是在传统的 MapReduce 计算框 架的基础上，利用其计算过程的优化，从而大大加快了数据分析、挖掘的运行和读写速度，并将计算单元缩小到更适合并行计算和重复使用的 RDD 计算模型。
- 机器学习中 ALS、凸优化梯度下降等。这些都需要基于数据集或者数据集的衍生数据 反复查询反复操作。MR 这种模式不太合适，即使多 MR 串行处理，性能和时间也是一个问题。数据的共享依赖于磁盘。另外一种是交互式数据挖掘，MR 显然不擅长。而 Spark 所基于的 scala 语言恰恰擅长函数的处理。
- Spark 是一个分布式数据快速分析项目。它的核心技术是弹性分布式数据集(Resilient Distributed Datasets)，提供了比 MapReduce 丰富的模型，可以快速在内存中对数据集 进行多次迭代，来支持复杂的数据挖掘算法和图计算算法。
- Spark 和 Hadoop 的根本差异是多个作业之间的数据通信问题 : Spark 多个作业之间数据通信是基于内存，而 Hadoop 是基于磁盘。
- Spark Task 的启动时间快。Spark 采用 fork 线程的方式，而 Hadoop 采用创建新的进程的方式。
- Spark 只有在 shuffle 的时候将数据写入磁盘，而 Hadoop 中多个 MR 作业之间的数据交互都要依赖于磁盘交互
- Spark 的缓存机制比 HDFS 的缓存机制高效。

经过上面的比较，我们可以看出在绝大多数的数据计算场景中，Spark 确实会比 MapReduce 更有优势。但是 Spark 是基于内存的，所以在实际的生产环境中，由于内存的限制，可能会 由于内存资源不够导致 Job 执行失败，此时，MapReduce 其实是一个更好的选择，所以 Spark 并不能完全替代 MR。

## Spark 的核心模块

![1613719473289.jpg](asset/1613719473289.jpg)

- Spark Core

    Spark Core 中提供了 Spark 最基础与最核心的功能，Spark 其他的功能如:Spark SQL，Spark Streaming，GraphX, MLlib 都是在 Spark Core 的基础上进行扩展的 
- Spark SQL

    Spark SQL 是 Spark 用来操作结构化数据的组件。通过 Spark SQL，用户可以使用 SQL 或者 Apache Hive 版本的 SQL 方言(HQL)来查询数据。
- Spark Streaming

    Spark Streaming 是 Spark 平台上针对实时数据进行流式计算的组件，提供了丰富的处理 数据流的 API。
- Spark MLlib

    MLlib 是 Spark 提供的一个机器学习算法库。MLlib 不仅提供了模型评估、数据导入等额外的功能，还提供了一些更底层的机器学习原语。
- Spark GraphX

    GraphX 是 Spark 面向图计算提供的框架与算法库。